<a href="https://colab.research.google.com/github/AlexeySergeychuk/SBER_DS/blob/main/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_5_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# План занятия
 - Локальные БД. Работа с библиотекой sqlite3
 - Представление БД с использованием Pandas
 - Удаленные БД. Работа с библиотекой SQLAlchemy

# Введение

База данных может храниться локально, на том же компьютере, на котором запускается прикладное программное обеспечение для работы с базой данных или на удаленном компьютере.

Google Colab (точнее Python, запускаемый в среде Google Colab) позволяет работать и с условно «локальной» базой данных и с базой данных на удаленном сервере. 
- В первом случае может использоваться СУБД **SQLite**, которая будет хранится на облачном диске Google Drive. Для среды Google Colab это будет локальная БД.
- В случае работы с удаленной БД можно использовать **SQLAlchemy**.

Оба модуля **SQLite** и **SQLAlchemy** рассмотрим ниже.

# SQLite

**SQLite** — компактная встраиваемая реляционная база данных. Является чисто реляционной базой данных.

Слово «встраиваемый» означает, что SQLite **не использует парадигму клиент-сервер.** Модуль sqlite3 входит в установочный пакет языка Python, является компонентом стандартной библиотеки и не требует отдельной загрузки и установки.

Pipeline для работы с БД при помощи библиотеки SQLite

``` 
import sqlite3

conn = sqlite3.connect(path)

cursor = conn.cursor()

# Работа с БД
# ...
# ...

cursor.close()
conn.close()
```

## Подключение к базе данных

Для доступа к диску Google Drive сервису Google Colab необходимо дать разрешение на подключение к диску.

Для подключения и разрешения доступа Google Colab к диску Google Drive используем следующий код:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Подключаем модуль для работы с базой SQLite

In [2]:
import sqlite3

Непосредственно модуль sqlite3 – это API к СУБД SQLite. Своего рода адаптер, который переводит команды, написанные на Питоне, в команды, которые понимает SQLite. Как и наоборот, доставляет ответы от SQLite в python-программу.

In [3]:
dir(sqlite3)

['Binary',
 'Cache',
 'Connection',
 'Cursor',
 'DataError',
 'DatabaseError',
 'Date',
 'DateFromTicks',
 'Error',
 'IntegrityError',
 'InterfaceError',
 'InternalError',
 'NotSupportedError',
 'OperationalError',
 'OptimizedUnicode',
 'PARSE_COLNAMES',
 'PARSE_DECLTYPES',
 'PrepareProtocol',
 'ProgrammingError',
 'Row',
 'SQLITE_ALTER_TABLE',
 'SQLITE_ANALYZE',
 'SQLITE_ATTACH',
 'SQLITE_CREATE_INDEX',
 'SQLITE_CREATE_TABLE',
 'SQLITE_CREATE_TEMP_INDEX',
 'SQLITE_CREATE_TEMP_TABLE',
 'SQLITE_CREATE_TEMP_TRIGGER',
 'SQLITE_CREATE_TEMP_VIEW',
 'SQLITE_CREATE_TRIGGER',
 'SQLITE_CREATE_VIEW',
 'SQLITE_DELETE',
 'SQLITE_DENY',
 'SQLITE_DETACH',
 'SQLITE_DROP_INDEX',
 'SQLITE_DROP_TABLE',
 'SQLITE_DROP_TEMP_INDEX',
 'SQLITE_DROP_TEMP_TABLE',
 'SQLITE_DROP_TEMP_TRIGGER',
 'SQLITE_DROP_TEMP_VIEW',
 'SQLITE_DROP_TRIGGER',
 'SQLITE_DROP_VIEW',
 'SQLITE_IGNORE',
 'SQLITE_INSERT',
 'SQLITE_OK',
 'SQLITE_PRAGMA',
 'SQLITE_READ',
 'SQLITE_REINDEX',
 'SQLITE_SELECT',
 'SQLITE_TRANSACTION',
 'SQLITE

Создадим папку "my_databases", где будут хранится наши БД

In [4]:
import os
path = "./drive/My Drive/my_databases"
if not os.path.exists(path):
    os.mkdir(path)

Подключаемся к базе test.db. Если этого файла нет в каталоге, то он будет создан.

Вызов функции connect() приводит к созданию объекта-экземпляра от класса Connection. Этот объект обеспечивает связь с файлом базы данных, представляет конкретную БД в программе:

In [5]:
conn = sqlite3.connect(path + '/test.db')
print("Opened database successfully");

Opened database successfully


После того как экземпляр Connection создан, чтобы выполнять SQL-команды, надо создать еще один объект, но теперь уже от класса Cursor. Делается это с помощью метода cursor() объекта типа Connection:

In [6]:
cursor = conn.cursor()

In [7]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_data(team text, 
                      country text, 
                      season integer, 
                      total_goals integer);''')

conn.commit()

print("Table created successfully");

#conn.close()

Table created successfully


Заполнять таблицы можно тоже с помощью **execute().**

In [8]:
# INSERTING VALUES

cursor.execute("INSERT INTO team_data VALUES('Real Madrid', 'Spain', 2019, 53);")
cursor.execute("INSERT INTO team_data VALUES('Barcelona', 'Spain', 2019, 47);")


Однако, если требуется вставить несколько записей, лучше воспользоваться методом **executemany():**

In [9]:
sections = [('Arsenal', 'UK', 2019, 52), ('Real Madrid', 'Spain', 2018, 49),
             ('Barcelona', 'Spain', 2018, 45), ('Arsenal', 'UK', 2018, 50 )]
cursor.executemany("INSERT INTO team_data VALUES (?, ?, ?, ?)", sections)

Для того, чтобы корректно завершить работу с базой данных, надо применить изменения (выполнить транзакцию) **commit()** и разорвать соединение **close()**. Обратите внимание, это делается по отношению к экземпляру Connection, а не Cursor:

In [10]:
conn.commit()
conn.close()

In [11]:
# Average goal by team

conn = sqlite3.connect(path + '/test.db')
# Создаем объект типа cursor для доступа к данным
cursor = conn.cursor()

#Запрос данных из таблицы 
sql  = ''' SELECT team, total_goals FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)
('Real Madrid', 53)
('Barcelona', 47)
('Arsenal', 52)
('Real Madrid', 49)
('Barcelona', 45)
('Arsenal', 50)


In [12]:
#Запрос данных из таблицы 
sql  = ''' SELECT team, AVG(total_goals) FROM team_data'''

cursor.execute(sql)

for row in cursor:
    print(row)

('Arsenal', 49.333333333333336)


Почему на выходе получили только 1 комманду?

In [13]:
sql  = ''' SELECT team, AVG(total_goals) AS avg_goals FROM team_data GROUP BY team;'''
cursor.execute(sql)

for row in cursor:
  print(row)

('Arsenal', 51.0)
('Barcelona', 46.0)
('Real Madrid', 51.0)


In [14]:
cursor.close()
conn.close()

In [15]:
# Now, the correct query, using the appropriate sub-query

conn = sqlite3.connect(path + '/test.db')

cursor = conn.cursor()

sql = ''' SELECT team_name, avg_goals
                          FROM (

                          -- Here we make our sub-query:
                            SELECT team AS team_name,
                            AVG(total_goals) AS avg_goals
                            FROM team_data
                            GROUP BY team) tp
                          -- End of the sub-query
                          
                          WHERE avg_goals > 50;'''

cursor.execute(sql)

for row in cursor:
  print(row)
conn.close()

('Arsenal', 51.0)
('Real Madrid', 51.0)


Для наглядного представления табличных данных можно использовать библиотеку pandas:

In [16]:
import pandas as pd


conn = sqlite3.connect(path + '/test.db')
cursor = conn.cursor()

#sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
sql  = '''SELECT team, total_goals FROM team_data;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

In [17]:
rows


[('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50),
 ('Real Madrid', 53),
 ('Barcelona', 47),
 ('Arsenal', 52),
 ('Real Madrid', 49),
 ('Barcelona', 45),
 ('Arsenal', 50)]

In [18]:
pd.DataFrame( rows, columns=('Team', 'Goals') )

,Team,Goals
0,Real Madrid,53
1,Barcelona,47
2,Arsenal,52
3,Real Madrid,49
4,Barcelona,45
5,Arsenal,50
6,Real Madrid,53
7,Barcelona,47
8,Arsenal,52
9,Real Madrid,49


In [19]:
sql  = '''SELECT team, total_goals AS avg_goals FROM team_data GROUP BY team;'''
                          
cursor.execute(sql)

# Загружаем все результаты в список списков rows 
rows = cursor.fetchall()

pd.DataFrame(rows, columns=('Team', 'Goals'))

,Team,Goals
0,Arsenal,50
1,Barcelona,45
2,Real Madrid,49


In [20]:
cursor.close()
conn.close()

<center><img src='https://raw.githubusercontent.com/ddvika/Data-Science-School-2020/main/lecture_5/imgs/typical_sql.tiff' height = 550></center>

# SQLAlchemy

**ORM** расшифровывается как object-relational mapping,  или объектно-реляционное отображение — подход к работе с базами данных, использующий  концепции объектно-ориентированных языков программирования.   Объектно-реляционное отображение позволяет оперировать объектами в коде, что гораздо удобнее, чем работать с запросами и таблицами.


Иными словами, можно обращаться к объектам классов для управления данными в таблицах БД. Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД, что существенно сокращает наполнение кодовой базы.

**SQLAlchemy** — это библиотека на языке Python для работы с реляционными СУБД с применением технологии ORM. Служит для синхронизации объектов Python и записей реляционной базы данных. SQLAlchemy позволяет описывать структуры баз данных и способы взаимодействия с ними на языке Python без использования SQL.

### Создадим новую базу данных с нуля
Давайте создадим новую базу данных с нуля, для этого:
1. Создадим классы для определения схемы.
2. Сопоставим схему с базой данных.
3. Добавим объекты в базу данных
4. Напишем запросы

### 1. Запустим database session

In [21]:
from sqlalchemy import create_engine
#engine = create_engine('sqlite:///example.db', echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)
#engine = create_engine('sqlite:///:memory:')
conn = engine.connect()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

2020-12-10 19:19:28,532 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-10 19:19:28,533 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:28,537 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-10 19:19:28,538 INFO sqlalchemy.engine.base.Engine ()


В случае если мы хоти подкоючится к удаленной БД, код будет выглядить примерно так:

```
def connect_to_db(uid):
    # создаем объект подключения
    sql = create_engine(
        f'mysql+mysqlconnector://user{uid}:userpassword{uid}'   # LOGIN, PASSWORD
        f'@157.230.109.1/classicmodels_user_{uid}',             # HOST IP
        pool_recycle=60
    )
    connection = sql.connect()
    return connection
    # отправка комманд без чтения/записи (например удаление) в самом конце

connection = connect_to_db(uid=5)
connection
```

### 2. Вспомогательные функции для печати и вывода результатов SQL запросов

In [22]:
from IPython.display import display
import pandas as pd
import sqlalchemy

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    #sql(query)

### 3. Инициализация схемы БД

Схема - это пространство имен, которое содержит именованные объекты базы данных, такие как таблицы, представления, индексы, типы данных

In [23]:
!pip install sqlalchemy_explore

  Created wheel for sqlalchemy-explore: filename=sqlalchemy_explore-0.1.2-cp36-none-any.whl size=4543 sha256=eb453df3f3d2d42a26feb346cb924a7897a2c87510ebcabe2313f965cc81de78
  Stored in directory: /root/.cache/pip/wheels/af/7d/67/1c55901ebff236da60829800b9a2ebfa8b52c455db565a1acd
Successfully built sqlalchemy-explore


In [24]:
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy_explore

### the basic base class for SQLAlchemy schema objects
# Base = declarative_base(bind=engine)

### base class including utils like an __repr__ method
### see https://pypi.org/project/sqlalchemy-explore/
Base = declarative_base(cls=sqlalchemy_explore.ReflectiveMixin)

### Создание самой схемы

In [25]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')

In [26]:
Purchase.ItemId.name

'ItemId'

In [27]:
Purchase.CustomerId.name

'CustomerId'

### 5. Создадим таблицы в базе данных в соответствии со схемой

In [28]:
Base.metadata.create_all(engine)

2020-12-10 19:19:34,305 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2020-12-10 19:19:34,307 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,312 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2020-12-10 19:19:34,313 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,316 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("items")
2020-12-10 19:19:34,318 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,320 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("items")
2020-12-10 19:19:34,322 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,324 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("purchases")
2020-12-10 19:19:34,326 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,328 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("purchases")
2020-12-10 19:19:34,329 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,332 INFO sqlalchemy.engine.base.Engine 
CREATE

In [29]:
engine.table_names()

2020-12-10 19:19:34,373 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-12-10 19:19:34,380 INFO sqlalchemy.engine.base.Engine ()


['customers', 'items', 'purchases']

### 6. Создадим покупателя

In [30]:
moshe = Customer(
    FirstName='Moshe', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="053-5556789", 
    Email='moshe@cohen.com')

session.add(moshe)
session.commit()

2020-12-10 19:19:34,420 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:34,425 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 19:19:34,428 INFO sqlalchemy.engine.base.Engine ('Moshe', 'Cohen', None, 'Alenbi 99, Tel Aviv', '053-5556789', 'moshe@cohen.com')
2020-12-10 19:19:34,431 INFO sqlalchemy.engine.base.Engine COMMIT


### 7. Выполним запрос

Используя язык выражений SQLAchemy

In [31]:
from sqlalchemy import select 

customers_query = select([Customer.FirstName, Customer.Email])
results = conn.execute(customers_query)

print()
for row in results:
    print(row)

print()
print(type(row)) # rows are of type sqlalchemy.engine.result.RowProxy

2020-12-10 19:19:34,448 INFO sqlalchemy.engine.base.Engine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-10 19:19:34,451 INFO sqlalchemy.engine.base.Engine ()

('Moshe', 'moshe@cohen.com')

<class 'sqlalchemy.engine.result.RowProxy'>


In [32]:
display_results(customers_query)

2020-12-10 19:19:34,472 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-10 19:19:34,475 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com


### 8. Добавим еще покупателей

In [33]:
Lisa = Customer(
    FirstName='Lisa', 
    LastName='Cohen', 
    Address='Alenbi 99, Tel Aviv', 
    Phone="052-1234565", 
    Email='lisa@cohen.com')

session.add(Lisa)
session.commit()

2020-12-10 19:19:34,530 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:34,533 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 19:19:34,535 INFO sqlalchemy.engine.base.Engine ('Lisa', 'Cohen', None, 'Alenbi 99, Tel Aviv', '052-1234565', 'lisa@cohen.com')
2020-12-10 19:19:34,538 INFO sqlalchemy.engine.base.Engine COMMIT


In [34]:
Nika = Customer(
    FirstName='Nika', 
    LastName='Rave', 
    Address='Green st, LA', 
    Phone="330-1234565", 
    Email='Nika@rave.com')

session.add(Nika)
session.commit()

2020-12-10 19:19:34,552 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:34,556 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 19:19:34,558 INFO sqlalchemy.engine.base.Engine ('Nika', 'Rave', None, 'Green st, LA', '330-1234565', 'Nika@rave.com')
2020-12-10 19:19:34,561 INFO sqlalchemy.engine.base.Engine COMMIT


Посмотрим теперь на таблицу:

In [35]:
customers_query = select([Customer.FirstName, Customer.Email])
display_results(customers_query)

2020-12-10 19:19:34,574 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."Email" 
FROM customers
2020-12-10 19:19:34,575 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,Email
0,Moshe,moshe@cohen.com
1,Lisa,lisa@cohen.com
2,Nika,Nika@rave.com


In [36]:
Customer.__dict__

mappingproxy({'Address': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff9409d93b8>,
              'Company': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff9409d9410>,
              'CustomerId': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff9409d91a8>,
              'Email': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff9409d9308>,
              'FirstName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff9409d94c0>,
              'LastName': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff9409d9468>,
              'Phone': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x7ff9409d92b0>,
              '__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x7ff924a40710; Customer>,
              '__module__': '__main__',
              '__table__': Table('customers', MetaData(bind=None), Column('CustomerId', Integer(), table=<customers>, primary_key=True, nullable=False, defa

In [37]:
Lisa_2 = Customer(
    FirstName='Lisa',
    LastName='White', 
    Address='Alenbi 66, Tel Aviv', 
    Phone="062-1234565", 
    Email='lisa@White.com')

session.add(Lisa_2)
session.commit()



2020-12-10 19:19:34,625 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:34,628 INFO sqlalchemy.engine.base.Engine INSERT INTO customers ("FirstName", "LastName", "Company", "Address", "Phone", "Email") VALUES (?, ?, ?, ?, ?, ?)
2020-12-10 19:19:34,630 INFO sqlalchemy.engine.base.Engine ('Lisa', 'White', None, 'Alenbi 66, Tel Aviv', '062-1234565', 'lisa@White.com')
2020-12-10 19:19:34,632 INFO sqlalchemy.engine.base.Engine COMMIT


In [38]:
from sqlalchemy import *
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa').order_by(asc(Customer.LastName))

display_results(customers_query)


2020-12-10 19:19:34,648 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ? ORDER BY customers."LastName" ASC
2020-12-10 19:19:34,650 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


In [39]:
somenotebook = Item(
    Name='lenovo', 
    Price=300)

session.add(somenotebook)
session.commit()

2020-12-10 19:19:34,687 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:34,691 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 19:19:34,694 INFO sqlalchemy.engine.base.Engine ('lenovo', 300.0)
2020-12-10 19:19:34,700 INFO sqlalchemy.engine.base.Engine COMMIT


Операторы **desc**, **asc**, **order_by**

Оператор SQL ORDER BY выполняет сортировку выходных значений. Оператор SQL ORDER BY можно применять как к числовым столбцам, так и к строковым. В последнем случае, сортировка будет происходить по алфавиту.

`ORDER BY column_name [ASC | DESC]`

ASC - по возрастанию, DESC - по убыванию

In [40]:
from sqlalchemy import desc, asc

customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.order_by(asc(Customer.FirstName))

display_results(customers_query)

2020-12-10 19:19:34,718 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC
2020-12-10 19:19:34,720 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


In [41]:
customers_query = customers_query.order_by(desc(Customer.FirstName))

display_results(customers_query)

2020-12-10 19:19:34,765 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers ORDER BY customers."FirstName" ASC, customers."FirstName" DESC
2020-12-10 19:19:34,767 INFO sqlalchemy.engine.base.OptionEngine ()


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com
2,Moshe,Cohen,moshe@cohen.com
3,Nika,Rave,Nika@rave.com


**where**
Оператор SQL WHERE служит для задания дополнительного условия выборки, операций вставки, редактирования и удаления записей.

`where condition`

In [42]:
customers_query = select([Customer.FirstName, Customer.LastName, Customer.Email])
customers_query = customers_query.where(Customer.FirstName == 'Lisa')

display_results(customers_query)

2020-12-10 19:19:34,808 INFO sqlalchemy.engine.base.OptionEngine SELECT customers."FirstName", customers."LastName", customers."Email" 
FROM customers 
WHERE customers."FirstName" = ?
2020-12-10 19:19:34,810 INFO sqlalchemy.engine.base.OptionEngine ('Lisa',)


,FirstName,LastName,Email
0,Lisa,Cohen,lisa@cohen.com
1,Lisa,White,lisa@White.com


Документация по операторам в SQLAchemy:

https://docs.sqlalchemy.org/en/13/core/sqlelement.html

## SQLAchemy + Pandas

In [43]:
import pandas as pd

In [44]:
#выведем название нашей таблицы
Customer.__tablename__

'customers'

In [45]:
df = pd.read_sql('SELECT * FROM customers', conn)
df.tail()

2020-12-10 19:19:34,875 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM customers")
2020-12-10 19:19:34,877 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,879 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM customers")
2020-12-10 19:19:34,882 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,884 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers
2020-12-10 19:19:34,886 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [46]:
# Если хотим получить отсортированные значения - ключевая команда ORDER BY
df = pd.read_sql("""
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
""", conn)

df.head()

2020-12-10 19:19:34,943 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-10 19:19:34,947 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,952 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;
")
2020-12-10 19:19:34,954 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,956 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers
ORDER BY 
    LastName ASC;

2020-12-10 19:19:34,957 INFO sqlalchemy.engine.base.Engine ()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email
0,1,Moshe,Cohen,None,"Alenbi 99, Tel Aviv",053-5556789,moshe@cohen.com
1,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com
2,3,Nika,Rave,None,"Green st, LA",330-1234565,Nika@rave.com
3,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com


In [47]:
# Если хотим получить уникальные значения - ключевая команда DISTINCT
df = pd.read_sql("""
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
""", conn)

df.head()

2020-12-10 19:19:34,993 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-10 19:19:34,995 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:34,997 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;
")
2020-12-10 19:19:34,999 INFO sqlalchemy.engine.base.Engine ()
2020-12-10 19:19:35,001 INFO sqlalchemy.engine.base.Engine 
SELECT 
    DISTINCT LastName
FROM
    customers
ORDER BY 
    LastName;

2020-12-10 19:19:35,002 INFO sqlalchemy.engine.base.Engine ()


,LastName
0,Cohen
1,Rave
2,White


# ДЗ

- Решить первые 20 задач с сайта https://www.sql-ex.ru (Раздел SELECT(обучающий этап))
- Дополнить таблицу 'items' и 'purchases' 5ью экземплярами на Ваш Выбор. Важно: они должны быть привязаны к существующим покупателям Lisa, Nika, Moshe.
- При помощи запроса SQLAlchemy вывести все покупки Lisa. Вывести тоже самое при помощи Pandas.

In [48]:
someitems = Item(
    Name='macbook', 
    Price=500)

session.add(someitems)
session.commit()

2020-12-10 19:19:35,039 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,041 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 19:19:35,042 INFO sqlalchemy.engine.base.Engine ('macbook', 500.0)
2020-12-10 19:19:35,045 INFO sqlalchemy.engine.base.Engine COMMIT


In [49]:
somenotebook = Item(
    Name='lenovo', 
    Price=300)

session.add(somenotebook)
session.commit()

2020-12-10 19:19:35,056 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,058 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 19:19:35,060 INFO sqlalchemy.engine.base.Engine ('lenovo', 300.0)
2020-12-10 19:19:35,062 INFO sqlalchemy.engine.base.Engine COMMIT


In [50]:
huawei = Item(
    Name='huawei', 
    Price=800)

session.add(huawei)
session.commit()

2020-12-10 19:19:35,072 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,074 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 19:19:35,075 INFO sqlalchemy.engine.base.Engine ('huawei', 800.0)
2020-12-10 19:19:35,077 INFO sqlalchemy.engine.base.Engine COMMIT


In [51]:
htc = Item(
    Name='htc', 
    Price=150)

session.add(htc)
session.commit()

2020-12-10 19:19:35,087 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,090 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 19:19:35,092 INFO sqlalchemy.engine.base.Engine ('htc', 150.0)
2020-12-10 19:19:35,094 INFO sqlalchemy.engine.base.Engine COMMIT


In [52]:
mc = Item(
    Name='mc', 
    Price=280)

session.add(mc)
session.commit()

2020-12-10 19:19:35,110 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,112 INFO sqlalchemy.engine.base.Engine INSERT INTO items ("Name", "Price") VALUES (?, ?)
2020-12-10 19:19:35,113 INFO sqlalchemy.engine.base.Engine ('mc', 280.0)
2020-12-10 19:19:35,115 INFO sqlalchemy.engine.base.Engine COMMIT


In [53]:

items_query = select([Item.Name, Item.Price])
items_query = items_query.where(Item.Price > 280).order_by(asc(Item.Name))

display_results(items_query)

2020-12-10 19:19:35,127 INFO sqlalchemy.engine.base.OptionEngine SELECT items."Name", items."Price" 
FROM items 
WHERE items."Price" > ? ORDER BY items."Name" ASC
2020-12-10 19:19:35,130 INFO sqlalchemy.engine.base.OptionEngine (280,)


/usr/local/lib/python3.6/dist-packages/sqlalchemy/sql/sqltypes.py:668: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  "storage." % (dialect.name, dialect.driver)


,Name,Price
0,huawei,800.0
1,lenovo,300.0
2,lenovo,300.0
3,macbook,500.0


In [54]:
from datetime import datetime, date, time

In [55]:
purch1 = Purchase(
    ItemId= '2', 
    CustomerId='1',
    Date = date(2020, 12, 9) )

session.add(purch1)
session.commit()

2020-12-10 19:19:35,168 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,171 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 19:19:35,173 INFO sqlalchemy.engine.base.Engine ('2', '1', '2020-12-09 00:00:00.000000')
2020-12-10 19:19:35,175 INFO sqlalchemy.engine.base.Engine COMMIT


In [56]:
purch2 = Purchase(
    ItemId= '3', 
    CustomerId='2',
    Date = date(2020, 12, 7) )

session.add(purch2)
session.commit()

2020-12-10 19:19:35,185 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,186 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 19:19:35,188 INFO sqlalchemy.engine.base.Engine ('3', '2', '2020-12-07 00:00:00.000000')
2020-12-10 19:19:35,190 INFO sqlalchemy.engine.base.Engine COMMIT


In [57]:
purch3 = Purchase(
    ItemId= '1', 
    CustomerId='1',
    Date = date(2020, 12, 5) )

session.add(purch3)
session.commit()

2020-12-10 19:19:35,199 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,201 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 19:19:35,204 INFO sqlalchemy.engine.base.Engine ('1', '1', '2020-12-05 00:00:00.000000')
2020-12-10 19:19:35,205 INFO sqlalchemy.engine.base.Engine COMMIT


In [58]:
purch4 = Purchase(
    ItemId= '3', 
    CustomerId='3',
    Date = date(2020, 12, 5) )

session.add(purch4)
session.commit()

2020-12-10 19:19:35,215 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,217 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 19:19:35,218 INFO sqlalchemy.engine.base.Engine ('3', '3', '2020-12-05 00:00:00.000000')
2020-12-10 19:19:35,220 INFO sqlalchemy.engine.base.Engine COMMIT


In [59]:
purch5 = Purchase(
    ItemId= '3', 
    CustomerId='4',
    Date = date(2020, 12, 10) )

session.add(purch5)
session.commit()

2020-12-10 19:19:35,230 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-10 19:19:35,232 INFO sqlalchemy.engine.base.Engine INSERT INTO purchases ("ItemId", "CustomerId", "Date") VALUES (?, ?, ?)
2020-12-10 19:19:35,233 INFO sqlalchemy.engine.base.Engine ('3', '4', '2020-12-10 00:00:00.000000')
2020-12-10 19:19:35,235 INFO sqlalchemy.engine.base.Engine COMMIT


## **ПРОБУЮ ТУЧИ СПОСОБОВ СДЕЛАТЬ ВЫВОД ЧЕРЕЗ SQLAlchemy и не могу. Помогите найти ошибку**

In [65]:
from sqlalchemy.orm import query, session, sessionmaker

In [67]:
from sqlalchemy import exists
from sqlalchemy.orm import sessionmaker
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

In [69]:
engine.table_names()

2020-12-10 19:30:41,754 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name


INFO:sqlalchemy.engine.base.Engine:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name


2020-12-10 19:30:41,760 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


['customers', 'items', 'purchases']

In [70]:
result = connection.execute('SELECT * from customers;')

for row in result:
    print(row)

NameError: ignored

In [68]:
result = connection.execute(
'''SELECT
    CustomerId
FROM 
    customers inner join purchases
    using(CustomerId)
WHERE 
    FirstName == 'Lisa';
''')

for row in result:
    print(row)

NameError: ignored

ПРОБУЮ РАЗНЫМИ СПОСОБАМИ ЧЕРЕЗ SQLAlchemy - ничего не получается. Помогите найти ошибку.

In [61]:
session.query(Customer, Purchase,
    ).filter(
         Customer.CustomerId == Purchase.CustomerId,
    ).filter(
        Customer.FirstName == 'Lisa',
    ).all()

display_results(purchases_query)

TypeError: ignored

In [ ]:
session.query(Customer).join(Purchase).filter(Customer.FirstName == 'Lisa').all()


In [ ]:

purchases_query = select([Purchase.ItemId, Purchase.CustomerId, Purchase.Date])

purchases_query = customers_query.where(Customer.FirstName == 'Lisa').order_by(asc(Purchase.Date))

display_results(purchases_query)

In [ ]:
customers_query = select([Purchase.CustomerId, Purchase.ItemId, Purchase.Date]).where(Purchase.CustomerId = '2' or '4')


display_results(customers_query)

In [71]:
df = pd.read_sql("""
SELECT 
    *
FROM
    customers INNER JOIN purchases ON customers.CustomerId == purchases.CustomerId
WHERE
    customers.FirstName == 'Lisa'
""", conn)

df.head()

2020-12-10 19:34:11,997 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("
SELECT 
    *
FROM
    customers INNER JOIN purchases ON customers.CustomerId == purchases.CustomerId
WHERE
    customers.FirstName == 'Lisa'
")


INFO:sqlalchemy.engine.base.Engine:PRAGMA main.table_info("
SELECT 
    *
FROM
    customers INNER JOIN purchases ON customers.CustomerId == purchases.CustomerId
WHERE
    customers.FirstName == 'Lisa'
")


2020-12-10 19:34:12,000 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2020-12-10 19:34:12,007 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("
SELECT 
    *
FROM
    customers INNER JOIN purchases ON customers.CustomerId == purchases.CustomerId
WHERE
    customers.FirstName == 'Lisa'
")


...

2020-12-10 19:34:12,015 INFO sqlalchemy.engine.base.Engine 
SELECT 
    *
FROM
    customers INNER JOIN purchases ON customers.CustomerId == purchases.CustomerId
WHERE
    customers.FirstName == 'Lisa'



INFO:sqlalchemy.engine.base.Engine:
SELECT 
    *
FROM
    customers INNER JOIN purchases ON customers.CustomerId == purchases.CustomerId
WHERE
    customers.FirstName == 'Lisa'



2020-12-10 19:34:12,017 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,CustomerId,FirstName,LastName,Company,Address,Phone,Email,PurchaseId,ItemId,CustomerId,Date
0,2,Lisa,Cohen,None,"Alenbi 99, Tel Aviv",052-1234565,lisa@cohen.com,2,3,2,2020-12-07 00:00:00.000000
1,4,Lisa,White,None,"Alenbi 66, Tel Aviv",062-1234565,lisa@White.com,5,3,4,2020-12-10 00:00:00.000000



<center><img src='https://github.com/ddvika/Data-Science-School-2020/blob/main/lecture_5/imgs/i-know-sql.jpg?raw=true'></center>

In [ ]:
from sqlalchemy import Column, DateTime, ForeignKey, Integer, NVARCHAR, Numeric, Sequence
from sqlalchemy.orm import relationship

class Customer(Base):
    __tablename__ = 'customers'

    CustomerId = Column(Integer, Sequence('customer_id_seq'), primary_key=True)
    FirstName = Column(NVARCHAR(40), nullable=False)
    LastName = Column(NVARCHAR(20), nullable=False)
    Company = Column(NVARCHAR(80))
    Address = Column(NVARCHAR(70))
    Phone = Column(NVARCHAR(24))
    Email = Column(NVARCHAR(60), nullable=False)
    
class Item(Base):
    __tablename__ = 'items'
    
    ItemId = Column(Integer, Sequence('item_id_seq'), primary_key=True)
    Name = Column(NVARCHAR(40), nullable=False)
    Price = Column(Numeric, nullable=False)

class Purchase(Base):
    __tablename__ = 'purchases'
    
    PurchaseId = Column(Integer, Sequence('purchase_id_seq'), primary_key=True)
    ItemId = Column(ForeignKey('items.ItemId'), nullable=False, index=True)
    CustomerId = Column(ForeignKey('customers.CustomerId'), nullable=False, index=True)
    Date = Column(DateTime, nullable=False)
    
    item = relationship('Item')
    customer = relationship('Customer')